# Hemocyte Recruitment Analysis - Preprocessing 
This notebook takes abdomen and hemocyte segmented features extracted by QuPath and consolidates features from all samples into dataframes for downstream analysis. Further, sample names are mapped and XY coordinates of hemocytes are scaled relative to the abdomen centroid. 

## Import required packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os import path
from pathlib import Path
import glob
sns.set_style('white')

## Set up input/output directories and experiment info

In [2]:

base_path = Path("/Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/")
sample_list = pd.read_csv(base_path.joinpath("samplist.tsv"),sep="\t",header=0)
sample_mappings = pd.read_csv(base_path.joinpath("sample_mapping.tsv"),sep="\t",header=0)

# Paths to abdomen measurements and hemocyte detection measurements
am_path= base_path.joinpath('data','combined','annotation_measurements')
dm_path= base_path.joinpath('data','combined','detection_measurements')

outputs = base_path.joinpath('results','combined')




## Create a dataframe of abdomen features

In [3]:

def get_sample_condition(imagename,sample_mapping):
    sampleid = imagename[:2]
    condition = sample_mapping[sample_mapping['Sample #'] == int(sampleid)]
    return condition['Condition'].iloc[0]

all_annotation_measurement_files = glob.glob(path.join(am_path , "*.txt"))
am_list = []
for amf in all_annotation_measurement_files:
    df = pd.read_csv(amf, index_col=None, header=0, sep="\t")
    am_list.append(df)


abdomen_coordinates = pd.concat(am_list, axis=0, ignore_index=True)

## annotate each fly abdomen with experimental group info 
abdomen_coordinates['condition'] = abdomen_coordinates['Image'].apply(get_sample_condition,sample_mapping=sample_list)
abdomen_coordinates['condition'] = np.where(abdomen_coordinates['condition'] == 'C', 'Fed', abdomen_coordinates['condition'])
abdomen_coordinates['condition'] = np.where(abdomen_coordinates['condition'] == 'S', 'Starved', abdomen_coordinates['condition'])

abdomen_coordinates.columns = abdomen_coordinates.columns.str.replace('Centroid X µm', 'Centroid_X')
abdomen_coordinates.columns = abdomen_coordinates.columns.str.replace('Centroid Y µm', 'Centroid_Y')
print(abdomen_coordinates.shape)
abdomen_coordinates[:5]


(97, 17)


,Image,Name,Class,Parent,ROI,Centroid_X,Centroid_Y,Num Detections,Num Positive,Positive %,Num Positive per mm^2,Area µm^2,Perimeter µm,Object ID,Object type,Classification,condition
0,75C_6_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,441.41,350.34,38.0,38.0,100.0,210.03,180922.8,1648.0,NaN,NaN,NaN,Fed
1,60E_2_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,407.74,330.97,36.0,36.0,100.0,168.72,213374.3,1926.7,NaN,NaN,NaN,Fed
2,24E_F3_2_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,383.65,360.84,87.0,87.0,100.0,366.05,237669.9,1998.3,NaN,NaN,NaN,Fed
3,76B_1_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,404.40,297.69,115.0,115.0,100.0,723.23,159008.5,1629.7,NaN,NaN,NaN,Starved
4,76A_6_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,391.34,316.31,111.0,111.0,100.0,645.43,171978.8,1608.8,NaN,NaN,NaN,Starved


## Map Fly and Section IDs for each abdomen image

In [4]:
# update sample names in the image column to new names in the "fly_section" format 
def map_samples(imageid, sample_mapping_lookup):
    try:
        fly_section_id = sample_mapping_lookup[sample_mapping_lookup['old']==imageid]['new'].iloc[0]
        fly_id = sample_mapping_lookup[sample_mapping_lookup['old']==imageid]['new'].iloc[0].split("_")[0]
        section_id = sample_mapping_lookup[sample_mapping_lookup['old']==imageid]['new'].iloc[0].split("_")[1]
    except: 
        fly_section_id = "notfound"
        fly_id = "none"
        section_id = "none"
    return(pd.Series([fly_section_id,fly_id,section_id]))

abdomen_coordinates[['fly_section_ID','fly_ID','section_ID']] = abdomen_coordinates.apply(lambda x: map_samples(x.Image, sample_mapping_lookup=sample_mappings), axis=1) 

abdomen_coordinates

,Image,Name,Class,Parent,ROI,Centroid_X,Centroid_Y,Num Detections,Num Positive,Positive %,Num Positive per mm^2,Area µm^2,Perimeter µm,Object ID,Object type,Classification,condition,fly_section_ID,fly_ID,section_ID
0,75C_6_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,441.41,350.34,38.0,38.0,100.0,210.030,180922.8,1648.0,NaN,NaN,NaN,Fed,75_17,75,17
1,60E_2_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,407.74,330.97,36.0,36.0,100.0,168.720,213374.3,1926.7,NaN,NaN,NaN,Fed,60_22,60,22
2,24E_F3_2_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,383.65,360.84,87.0,87.0,100.0,366.050,237669.9,1998.3,NaN,NaN,NaN,Fed,1_2,1,2
3,76B_1_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,404.40,297.69,115.0,115.0,100.0,723.230,159008.5,1629.7,NaN,NaN,NaN,Starved,76_7,76,7
4,76A_6_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,391.34,316.31,111.0,111.0,100.0,645.430,171978.8,1608.8,NaN,NaN,NaN,Starved,76_6,76,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,72D_2_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,426.08,304.94,61.0,61.0,100.0,262.190,232652.9,1904.1,NaN,NaN,NaN,Fed,72_17,72,17
93,78_6_FITC.tif,NaN,NaN,Root object (Image),Polygon,470.27,1634.80,8.0,8.0,100.0,9.481,843801.3,4541.9,97f5c0c9-37f4-47a2-a176-dbe6ef362d18,Annotation,NaN,Starved,78_6,78,6
94,74_4_FITC.tif,NaN,NaN,Root object (Image),Polygon,433.14,1673.00,1.0,1.0,100.0,1.007,993189.2,4646.3,9ea94957-74bb-45ec-a2da-79dfe64fae20,Annotation,NaN,Fed,74_4,74,4
95,43_5_FITC.tif,NaN,NaN,Root object (Image),Polygon,460.91,1627.40,14.0,14.0,100.0,11.100,1261619.0,5761.9,cbc718b4-268c-4a7a-bfae-16604ea8a7a5,Annotation,NaN,Fed,43_5,43,5


## Create a dataframe of hemocyte features from all samples - scale coordinates relative to abdomen centroid and map fly and section IDs 

In [5]:
from scipy.spatial import distance
import math 
sns.set_palette("pastel")

def zero_scale_x(image, x, polygon):
    polygon_x = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_X'].iloc[0] 
    zero_scaled_x = x-polygon_x
    return(zero_scaled_x)

def zero_scale_y(image, y, polygon):
   polygon_y = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_Y'].iloc[0] 
   zero_scaled_y = y-polygon_y
   return(zero_scaled_y)

def get_centroid_distance(image,x,y,polygon):
    polygon_x = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_X'].iloc[0]
    polygon_y = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_Y'].iloc[0]
    centroid_distance = distance.euclidean([polygon_x, polygon_y], [x,y])

    return centroid_distance
    

all_detection_measurement_files = glob.glob(path.join(dm_path , "*.txt"))
dm_list = []

for dm in all_detection_measurement_files:
    #print(dm)
    detection_measurements = pd.read_csv(dm,sep="\t",header=0)
    detection_measurements.columns = detection_measurements.columns.str.replace('Centroid X µm', 'Centroid_X')
    detection_measurements.columns = detection_measurements.columns.str.replace('Centroid Y µm', 'Centroid_Y')

    try:
        detection_measurements["centroid_x_zeroscaled"] = detection_measurements.apply(
            lambda x: zero_scale_x(x.Image, x.Centroid_X,polygon=abdomen_coordinates), axis=1) 
        detection_measurements["centroid_y_zeroscaled"] = detection_measurements.apply(
            lambda x: zero_scale_y(x.Image, x.Centroid_Y,polygon=abdomen_coordinates), axis=1)
        detection_measurements["centroid_distances"] = detection_measurements.apply(
            lambda x: get_centroid_distance(x.Image, x.Centroid_X,x.Centroid_Y,polygon=abdomen_coordinates), axis=1)
    except ValueError:
        print(dm)

    dm_list.append(detection_measurements)
    
    
#print(dm_list)
hemocyte_coordinates = pd.concat(dm_list, axis=0, ignore_index=True)
hemocyte_coordinates[['fly_section_ID','fly_ID','section_ID']] = hemocyte_coordinates.apply(lambda x: map_samples(x.Image, sample_mapping_lookup=sample_mappings), axis=1) 

hemocyte_coordinates[:5]

/Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/data/combined/detection_measurements/81_8_FITC.txt


,Image,Name,Class,Parent,ROI,Centroid_X,Centroid_Y,Nucleus: Area,Nucleus: Perimeter,Nucleus: Circularity,...,Delaunay: Max triangle area,centroid_x_zeroscaled,centroid_y_zeroscaled,centroid_distances,Object ID,Object type,Classification,fly_section_ID,fly_ID,section_ID
0,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,431.16,152.82,5.4900,8.0218,1.0000,...,5813.8784,-10.25,-197.52,197.785775,NaN,NaN,NaN,75_17,75,17
1,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,444.58,156.58,11.3722,14.1223,0.7165,...,572.5013,3.17,-193.76,193.785930,NaN,NaN,NaN,75_17,75,17
2,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,452.20,156.48,19.9993,20.4010,0.6038,...,22.9957,10.79,-193.86,194.160047,NaN,NaN,NaN,75_17,75,17
3,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,465.84,157.54,14.1172,16.4980,0.6518,...,64.5575,24.43,-192.80,194.341619,NaN,NaN,NaN,75_17,75,17
4,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,455.32,163.11,9.4114,12.8552,0.7157,...,543.2628,13.91,-187.23,187.746001,NaN,NaN,NaN,75_17,75,17


## Write abdomen and hemocyte features to csv files for downstream analysis 

In [6]:
hemocyte_coordinates.to_csv(outputs.joinpath("hemocyte_coordinates_midsections.csv"), sep="\t",index=False)
abdomen_coordinates.to_csv(outputs.joinpath("abdomen_coordinates_midsections.csv"), sep="\t",index=False)